# SparkSQL
Spark SQL is a Spark module for structured data processing. Unlike the basic Spark RDD API, the interfaces provided by Spark SQL provide Spark with more information about the structure of both the data and the computation being performed. Internally, Spark SQL uses this extra information to perform extra optimizations.

![Sparkcomponents](../images/Sparkcomponents.jpg)

The entry point into all functionality in Spark is the SparkSession class through the SparkContext.

In [16]:
import os
# Cargamos la librería externa que nos permite abrir formatos .avro
# Para ello debemos descargarnos la librería decom.databricks_spark-avro
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /home/afernandez/.ivy2/jars/com.databricks_spark-avro_2.11-4.0.0.jar pyspark-shell'

In [17]:
import pyspark

conf = pyspark.SparkConf().setAppName('MiPrimeraSparkApp').setMaster('local[*]') #Creamos la configuración
sc = pyspark.SparkContext(conf = conf) #Abrimos el contexto de Spark

## 0 · Open SparkSQL Session

In [19]:
import pyspark.sql

sparkSession = pyspark.sql.SparkSession(sc, jsparkSession=None)

spark = sparkSession\
.builder\
.master("local")\
.appName("holi")\
.config("spark.some.config.option", "some-value")\
.getOrCreate()

## 1 · Crear Dataframes
A **Dataset** is a distributed collection of data. Dataset is a new interface added in Spark 1.6 that provides the benefits of RDDs (strong typing, ability to use powerful lambda functions) with the benefits of Spark SQL’s optimized execution engine. A Dataset can be constructed from JVM objects and then manipulated using functional transformations (map, flatMap, filter, etc.). The Dataset API is available in Scala and Java.  

A **DataFrame** is a Dataset organized into named columns. It is conceptually equivalent to a table in a relational database or a data frame in R/Python, but with richer optimizations under the hood.  
They can be constructed from a wide array of sources such as,
* Structured data files
* Tables in Hive
* External databases
* Existing RDDs
  
DataFrame is represented by a **Dataset of Rows**. In Scala it's a type alias of Dataset \[ Row \]. In Java, Dataset < Row > *(Java)*

### Structured Data Files

In [20]:
Distdata = sc.parallelize([(1,"DataScientists"),(2,"Developers"),(3,"Scrums")])

* If **schema** is ***None*** SparkSQL infers the schema from data.

In [21]:
DataFrameData = spark.createDataFrame(Distdata)
DataFrameData.show()

+---+--------------+
| _1|            _2|
+---+--------------+
|  1|DataScientists|
|  2|    Developers|
|  3|        Scrums|
+---+--------------+



* If **schema** is a ***list of column names***, the __type of each column__ will be inferred from data

In [22]:
DataFrameData = spark.createDataFrame(Distdata,["Importancia","Curro"])
DataFrameData.show()

+-----------+--------------+
|Importancia|         Curro|
+-----------+--------------+
|          1|DataScientists|
|          2|    Developers|
|          3|        Scrums|
+-----------+--------------+



* If **schema** is ***StructType***, which consists of ***a list of StructField*** with ***pyspark.sql.types objects***, those data types *must agree* with original data,else Python will throw an exception.

In [23]:
from pyspark.sql.types import *
schema = StructType([
    StructField("Importancia", IntegerType(), True),
    StructField("Curro", StringType(), True)])
DataFrameData = spark.createDataFrame(Distdata,schema)
DataFrameData.show()

+-----------+--------------+
|Importancia|         Curro|
+-----------+--------------+
|          1|DataScientists|
|          2|    Developers|
|          3|        Scrums|
+-----------+--------------+



### Comma Separated Values

In [24]:
clientes = spark.read.load("../data/clientes.csv",
                     format="csv", sep=",", inferSchema="true", header="false")
clientes.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)



In [25]:
schemaClientes = (StructType([
    StructField("DNI", IntegerType(), True),
    StructField("Nombre", StringType(), True),
    StructField("Address", StringType(), True)]))

In [26]:
clientsRDD = clientes.rdd
type(clientsRDD)

pyspark.rdd.RDD

In [29]:
clientesNew = spark.createDataFrame(clientsRDD,schemaClientes)
clientesNew.show()

+--------+--------------------+--------------------+
|     DNI|              Nombre|             Address|
+--------+--------------------+--------------------+
|80000000|Antonio Lopez Ram...|Calle Cantalapied...|
|70000000|Francisco Arias S...|Avenida de Americ...|
|50000000|Norberto Marias Q...|     Calle Uganda 88|
|10000000|Julio Cortazar Ca...|   Calle Bruselas 14|
|20000000| Arturo Belano Yañez|Travesia de Calvo...|
+--------+--------------------+--------------------+



### Apache Hadoop formats
#### parquet

In [12]:
usuarios = spark.read.parquet("../data/userdata1.parquet")
usuarios.take(1)

[Row(registration_dttm=datetime.datetime(2016, 2, 3, 8, 55, 29), id=1, first_name='Amanda', last_name='Jordan', email='ajordan0@com.com', gender='Female', ip_address='1.197.201.2', cc='6759521864920116', country='Indonesia', birthdate='3/8/1971', salary=49756.53, title='Internal Auditor', comments='1E+02')]

In [13]:
usuarios2 = spark.read.format("parquet").load("../data/userdata1.parquet")
usuarios2.take(1)

[Row(registration_dttm=datetime.datetime(2016, 2, 3, 8, 55, 29), id=1, first_name='Amanda', last_name='Jordan', email='ajordan0@com.com', gender='Female', ip_address='1.197.201.2', cc='6759521864920116', country='Indonesia', birthdate='3/8/1971', salary=49756.53, title='Internal Auditor', comments='1E+02')]

In [14]:
usuarios.printSchema()

root
 |-- registration_dttm: timestamp (nullable = true)
 |-- id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- ip_address: string (nullable = true)
 |-- cc: string (nullable = true)
 |-- country: string (nullable = true)
 |-- birthdate: string (nullable = true)
 |-- salary: double (nullable = true)
 |-- title: string (nullable = true)
 |-- comments: string (nullable = true)



#### Avro
In order to read this data format files, the appropiate .jar has to be previously loaded in the SparkContext. (See first command)

In [15]:
twitter = spark.read.format("com.databricks.spark.avro").load("../data/twitter.avro")
twitter.show()

+----------+--------------------+----------+
|  username|               tweet| timestamp|
+----------+--------------------+----------+
|    miguno|Rock: Nerf paper,...|1366150681|
|BlizzardCS|Works as intended...|1366154481|
+----------+--------------------+----------+



In [16]:
twitter2 = spark.read.load("../data/twitter.avro",
                     format="com.databricks.spark.avro")
twitter2.show()

+----------+--------------------+----------+
|  username|               tweet| timestamp|
+----------+--------------------+----------+
|    miguno|Rock: Nerf paper,...|1366150681|
|BlizzardCS|Works as intended...|1366154481|
+----------+--------------------+----------+



In [17]:
twitter.printSchema()

root
 |-- username: string (nullable = true)
 |-- tweet: string (nullable = true)
 |-- timestamp: long (nullable = true)



### pandas Import/Export

#### Ensure PyArrow installed
If you install PySpark using pip, then PyArrow can be brought in as an extra dependency of the SQL module with the command pip install pyspark[sql]. Otherwise, you must ensure that PyArrow is installed and available on all cluster nodes.

In [18]:
import numpy as np
import pandas as pd

# Capacitar la transferencia de datos columnares basados en Arrow
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
    
# Generar un DataFrame de pandas
pdf = pd.DataFrame(np.random.rand(100, 3))

# Crear un DataFrame de Spark a partir de Pandas a través de Arrow
df = spark.createDataFrame(pdf)

# Convertir el DataFrame de Spark de vuelta a Pandas a través de Arrow
result_pdf = df.select("*").toPandas()

result_pdf

,0,1,2
0,0.770794,0.468203,0.342891
1,0.053756,0.127453,0.989758
2,0.539427,0.278439,0.496780
3,0.832618,0.866899,0.239864
4,0.589955,0.530505,0.564471
5,0.068579,0.477961,0.505941
6,0.522635,0.281982,0.745112
7,0.263354,0.675405,0.167618
8,0.934501,0.097933,0.545483
9,0.077042,0.002963,0.094425


## 2 · DataFrames Operations

We can access data in a column by attribute **(df.age)** and by index **(df['age'])**. Better the last way.

In [19]:
rdd = sc.parallelize([("null","Michael"),(30,"Andy"),(19,"Justin"),(19,"Alejandra"),(19,"Justin")])
df = spark.createDataFrame(rdd,["age","name"])
df.printSchema()

root
 |-- age: string (nullable = true)
 |-- name: string (nullable = true)



In [20]:
df.select("age").show()

+----+
| age|
+----+
|null|
|  30|
|  19|
|  19|
|  19|
+----+



In [21]:
df.select(df['name'], df['age'] + 1).show()

+---------+---------+
|     name|(age + 1)|
+---------+---------+
|  Michael|     null|
|     Andy|     31.0|
|   Justin|     20.0|
|Alejandra|     20.0|
|   Justin|     20.0|
+---------+---------+



In [22]:
df.filter(df['age'] > 21).show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



In [23]:
print(df.distinct().count())
df.distinct().show()

4
+----+---------+
| age|     name|
+----+---------+
|  30|     Andy|
|  19|Alejandra|
|  19|   Justin|
|null|  Michael|
+----+---------+



In [24]:
df.groupBy("age").count().show()

+----+-----+
| age|count|
+----+-----+
|  30|    1|
|  19|    3|
|null|    1|
+----+-----+



In [25]:
df.select(df["name"], (df.age + 10)).show()
# La función DataFrame.select(*cols)
# *cols = lista de nombres de columna o expresiones (Columna)

+---------+----------+
|     name|(age + 10)|
+---------+----------+
|  Michael|      null|
|     Andy|      40.0|
|   Justin|      29.0|
|Alejandra|      29.0|
|   Justin|      29.0|
+---------+----------+



There is a library plenty of **functions** designed for DataFrames formats, arithmetics... , called **pyspark.sql.functions**.

In [26]:
import pyspark.sql.functions as funciones
df.select(funciones.avg("age")
          .alias("media")
         ).show()

+-----+
|media|
+-----+
|21.75|
+-----+



In [27]:
df2 = df.select(df["name"], (df.age + 0.74).alias("new_age"))
df2.show()

+---------+-------+
|     name|new_age|
+---------+-------+
|  Michael|   null|
|     Andy|  30.74|
|   Justin|  19.74|
|Alejandra|  19.74|
|   Justin|  19.74|
+---------+-------+



In [28]:
import pyspark.sql.functions as funciones
df2.select(funciones.ceil("new_age").alias("approximate_age")).show()

+---------------+
|approximate_age|
+---------------+
|           null|
|             31|
|             20|
|             20|
|             20|
+---------------+



## 3 · Running SQL Queries Programmatically
To run SQL queries in a DataFrame, we have to create a **temporal view** in the SparkSQL session.

In [29]:
df2.createOrReplaceTempView("people")

sqlDF = spark.sql("SELECT * FROM people")
sqlDF.show()

+---------+-------+
|     name|new_age|
+---------+-------+
|  Michael|   null|
|     Andy|  30.74|
|   Justin|  19.74|
|Alejandra|  19.74|
|   Justin|  19.74|
+---------+-------+



In [30]:
sql2DF = spark.sql("SELECT name FROM people WHERE new_age<20")
sql2DF.distinct().show()

+---------+
|     name|
+---------+
|Alejandra|
|   Justin|
+---------+



### Running SQL Queries In Files

In [31]:
SQLparquetQuery = spark.sql("SELECT * FROM parquet.`../data/userdata1.parquet`")
SQLparquetQuery.show(2)

+-------------------+---+----------+---------+----------------+------+--------------+----------------+---------+---------+---------+----------------+--------+
|  registration_dttm| id|first_name|last_name|           email|gender|    ip_address|              cc|  country|birthdate|   salary|           title|comments|
+-------------------+---+----------+---------+----------------+------+--------------+----------------+---------+---------+---------+----------------+--------+
|2016-02-03 08:55:29|  1|    Amanda|   Jordan|ajordan0@com.com|Female|   1.197.201.2|6759521864920116|Indonesia| 3/8/1971| 49756.53|Internal Auditor|   1E+02|
|2016-02-03 18:04:03|  2|    Albert|  Freeman| afreeman1@is.gd|  Male|218.111.175.34|                |   Canada|1/16/1968|150280.17|   Accountant IV|        |
+-------------------+---+----------+---------+----------------+------+--------------+----------------+---------+---------+---------+----------------+--------+
only showing top 2 rows



## 4 · Global Temporary Views
Temporary views in Spark SQL are session-scoped and will disappear if the session that creates it terminates. If you want to have a temporary view that is shared among all sessions and keep alive until the Spark application terminates, you can create a global temporary view. **Global temporary view** is tied to a system preserved database **global_temp**, and we must use the qualified name to refer it, e.g. SELECT * FROM global_temp.view1.

In [32]:
df.createGlobalTempView("people")

In [33]:
spark.sql("SELECT * TemporaryFROM global_temp.people").show()

+----+---------+
| age|     name|
+----+---------+
|null|  Michael|
|  30|     Andy|
|  19|   Justin|
|  19|Alejandra|
|  19|   Justin|
+----+---------+



In [34]:
spark.newSession().sql("SELECT * FROM global_temp.people").show()

+----+---------+
| age|     name|
+----+---------+
|null|  Michael|
|  30|     Andy|
|  19|   Justin|
|  19|Alejandra|
|  19|   Justin|
+----+---------+



In [15]:
spark.stop()
sc.stop()